In [1]:
import cv2
import numpy as np

In [2]:
def show_images(imgList):
    while True:
        for (name,img) in imgList:
            cv2.imshow(name,img)
        pressed_key = cv2.waitKey(0)
        if pressed_key==27:
            break
    cv2.destroyAllWindows()

## Gauss Newton

In [3]:
def create_mask(shape,corners,winsize):
    mask = np.zeros(shape[:2],dtype=np.uint8)
    for corner in corners:
        cv2.rectangle(mask,tuple(corner-int(winsize/2)),tuple(corner+int(winsize/2)),255,-1)
    return mask

In [4]:
def get_derivativeof_warp_affine(x,y):
    return np.array([[x,0,y,0,1,0],[0,x,0,y,0,1]])

In [ ]:
def gauss_newton_prev(baseimg,endimg,mask,guess):
    assert (baseimg.shape==endimg.shape),"Template and Image should be of same shape"
    endimg = endimg.copy()
    #return baseimg
    
    ## Computing difference T(x)-I(W(x;p))
    transimg = cv2.warpAffine(baseimg,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    transmask = cv2.warpAffine(mask,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    diff = np.expand_dims(endimg-transimg,-1) ## (x,y,3,1)
    
    show_images([("target",endimg),("Trans img",transimg),("Warped mask",transmask)])
    
    
    ## Calculating gradient and theirs warps (3)
    sobelx = cv2.Sobel(baseimg,cv2.CV_64F,1,0,ksize=3) ## (x,y,3)
    sobely = cv2.Sobel(baseimg,cv2.CV_64F,0,1,ksize=3) ## (x,y,3)
    
    show_images([("gradx",sobelx),("grady",sobely)])
    
    warped_gradx = cv2.warpAffine(sobelx,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    warped_grady = cv2.warpAffine(sobely,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    warped_grad = np.stack([warped_gradx,warped_grady],-1) ## (x,y,3,2)
    
    show_images([("Warped gradx",warped_gradx),("Warped grady",warped_grady)])
    
    
    ## Caculation Jacobian (4)
    J = np.zeros((baseimg.shape[0],baseimg.shape[1],2,6)) ## (x,y,2,6)
    for x in range(J.shape[0]):
        for y in range(J.shape[1]):
            J[x,y] = get_derivativeof_warp_affine(y,x)
#             J[x,y] = get_derivativeof_warp_affine(x,y) ## THIS SHOULD NOT WORK

    ## Computing steepest descent images (5)
    steep_desc = np.matmul(warped_grad,J)  ## (x,y,3,6)
    steep_desc[transmask==0] = 0
    steep_desc_trans = steep_desc.transpose((0,1,3,2)) ## (x,y,6,3)
    
    ## Computing hessian (6)
    hessian = np.sum(np.sum(np.matmul(steep_desc_trans,steep_desc),axis=0),axis=0) #(6,6)
    
    ## Computing other term (7)
    other_term = np.sum(np.sum(np.matmul(steep_desc_trans,diff),axis=0),axis=0) #(6,1)
    
    ## calc update param (8)
    inv_hess = np.linalg.inv(hessian) ## (6,6)
    del_p = np.matmul(inv_hess,other_term) ## (6,1)
    
    return np.transpose(del_p.reshape([3,2]))

In [5]:
def gauss_newton_iter(baseimg,endimg,mask,guess):
    assert (baseimg.shape==endimg.shape),"Template and Image should be of same shape"
    endimg = endimg.copy()
    shape = baseimg.shape
    #return baseimg
    
#     show_images([("Input",baseimg),("Output",endimg),("mask",mask)])
    
    ## Computing difference T(x)-I(W(x;p))
    transimg = cv2.warpAffine(baseimg,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,c)
    transimg = transimg.reshape(shape)    
    transmask = cv2.warpAffine(mask,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,c)
    endimg[transmask==0]=0
    masked_trans_img = transimg[transmask==255] ## (m,c)
    masked_end_img = endimg[transmask==255]  ##(m,c)
#     print("Shaped of masked trans img: ",masked_trans_img.shape)
    
#     show_images([("target",endimg),("Trans img",transimg),("Warped mask",transmask)])
    
#     diff = np.expand_dims(endimg-transimg,-1) ## (x,y,c,1)
    diff = np.expand_dims(masked_end_img-masked_trans_img,-1) ## (m,c,1)
    
    
    ## Calculating gradient and theirs warps (3)
    sobelx = cv2.Sobel(baseimg,cv2.CV_64F,1,0,ksize=3) ## (x,y,c)
    sobely = cv2.Sobel(baseimg,cv2.CV_64F,0,1,ksize=3) ## (x,y,c)
    
#     show_images([("gradx",sobelx),("grady",sobely)])
    
    warped_gradx = cv2.warpAffine(sobelx,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,c)
    warped_gradx = warped_gradx.reshape(shape)
    warped_grady = cv2.warpAffine(sobely,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,c)
    warped_grady = warped_grady.reshape(shape)
    
#     show_images([("Warped gradx",warped_gradx),("Warped grady",warped_grady)])
    
    warped_grad = np.stack([warped_gradx,warped_grady],-1) ## (x,y,c,2)
    ## masked grad
    warped_grad = warped_grad[transmask==255]  ##(m,c,2)
    
    ## Caculation Jacobian (4)
    J = np.zeros((baseimg.shape[0],baseimg.shape[1],2,6)) ## (x,y,2,6)
    for x in range(J.shape[0]):
        for y in range(J.shape[1]):
            J[x,y] = get_derivativeof_warp_affine(y,x)
    ## masked jacobian
    J = J[transmask==255]  ##(m,2,6)
    
    ## Computing steepest descent images (5)
#     steep_desc = np.matmul(warped_grad,J)  ## (x,y,c,6)
    steep_desc = np.matmul(warped_grad,J)  ## (m,c,6)
#     steep_desc[transmask==0] = 0
#     steep_desc_trans = steep_desc.transpose((0,1,3,2)) ## (x,y,6,c)
    steep_desc_trans = steep_desc.transpose((0,2,1)) ## (m,6,c)
    
    ## Computing hessian (6)
#     hessian = np.sum(np.sum(np.matmul(steep_desc_trans,steep_desc),axis=0),axis=0) #(6,6)
    hessian = np.sum(np.matmul(steep_desc_trans,steep_desc),axis=0) #(6,6)
    
    ## Computing other term (7)
#     other_term = np.sum(np.sum(np.matmul(steep_desc_trans,diff),axis=0),axis=0) #(6,1)
    other_term = np.sum(np.matmul(steep_desc_trans,diff),axis=0) #(6,1)
    
    ## calc update param (8)
    inv_hess = np.linalg.inv(hessian) ## (6,6)
    del_p = np.matmul(inv_hess,other_term) ## (6,1)
    
    return np.transpose(del_p.reshape([3,2]))

In [ ]:
def get_transform(baseimg,endimg,corners):
    baseimg = baseimg.copy()
    endimg = endimg.copy()
    
    # Convert images to grayscale
    basegray = np.expand_dims(cv2.cvtColor(baseimg,cv2.COLOR_BGR2GRAY),-1)
    endgray = np.expand_dims(cv2.cvtColor(endimg,cv2.COLOR_BGR2GRAY),-1)
    mask = create_mask(basegray.shape,corners,5)
    
    transform = np.eye(2,3,dtype=np.float32)
    
    while(True):
        change_param = gauss_newton_iter(baseimg,endimg,mask,transform)
        transform += change_param
        norm = np.linalg.norm(change_param)
        if (norm < 1e-6):
            break
    
    return transform

In [10]:
baseimg = right.copy()
endimg = left.copy()
corners = corners.copy()

# Convert images to grayscale
# basegray = np.expand_dims(cv2.cvtColor(baseimg,cv2.COLOR_BGR2GRAY),-1)
# endgray = np.expand_dims(cv2.cvtColor(endimg,cv2.COLOR_BGR2GRAY),-1)
basegray = baseimg
endgray = endimg
mask = create_mask(basegray.shape,corners,5)

transform = np.eye(2,3,dtype=np.float32)

In [11]:
# mask[:]=255

In [12]:
basegray.shape

(480, 640, 3)

In [13]:
#transform=best_param.copy()
# transform=my_param.copy()

In [14]:
transform

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.]], dtype=float32)

In [21]:
for i in range(500):
    change_param = gauss_newton_iter(basegray,endgray,mask,transform)
#     change_param = gauss_newton_prev(basegray,endgray,mask,transform)
    transform += change_param
    print(i)
    print(np.linalg.norm(change_param))
    print(transform)

0
0.560101925159
[[  1.06300104e+00   4.54342999e-02  -2.65912418e+01]
 [  2.24184021e-02   9.80080605e-01  -3.24484611e+00]]
1
0.483228286436
[[  1.06408870e+00   4.62901480e-02  -2.70602226e+01]
 [  2.30510645e-02   9.79728580e-01  -3.36131048e+00]]
2
0.472701331228
[[  1.06516850e+00   4.71611321e-02  -2.75190887e+01]
 [  2.36888975e-02   9.79315639e-01  -3.47482586e+00]]
3
0.460854526816
[[  1.06619298e+00   4.80110534e-02  -2.79584198e+01]
 [  2.42997892e-02   9.78930235e-01  -3.61401415e+00]]
4
0.434192710665
[[  1.06721401e+00   4.86434400e-02  -2.83427181e+01]
 [  2.49641929e-02   9.78738010e-01  -3.81609464e+00]]
5
0.47779371192
[[  1.06823063e+00   4.93631177e-02  -2.87495861e+01]
 [  2.57395804e-02   9.78566170e-01  -4.06657982e+00]]
6
0.517398805565
[[  1.06934297e+00   5.01159169e-02  -2.91808338e+01]
 [  2.65762992e-02   9.78528380e-01  -4.35245514e+00]]
7
0.535939969909
[[  1.07034481e+00   5.08389995e-02  -2.95694523e+01]
 [  2.74956152e-02   9.78788733e-01  -4.72151470

77
0.532030697903
[[  1.14429283   0.10788143 -60.25736618]
 [  0.07366633   0.94925964 -13.54154396]]
78
0.591711332511
[[  1.1454277    0.10917003 -60.83224869]
 [  0.07428469   0.94884568 -13.68165112]]
79
0.557381725188
[[  1.14656496   0.11031689 -61.38022232]
 [  0.07494352   0.94828522 -13.78361225]]
80
0.513850765778
[[  1.14765596   0.1113125  -61.87055969]
 [  0.07562679   0.9479202  -13.93726921]]
81
0.471291931904
[[  1.14868498   0.11221212 -62.32587433]
 [  0.07628456   0.94749302 -14.05893421]]
82
0.480280336201
[[  1.14971983   0.11315893 -62.78224945]
 [  0.07702374   0.94709617 -14.20856476]]
83
0.547413814392
[[  1.15086412   0.1142206  -63.29878616]
 [  0.07785045   0.94681144 -14.38980579]]
84
0.53345977277
[[  1.15195668   0.11532927 -63.81855011]
 [  0.07854313   0.94644594 -14.50990009]]
85
0.515046420113
[[  1.15309691   0.1163566  -64.33293152]
 [  0.07907648   0.94585693 -14.53605938]]
86
0.490014050598
[[  1.15420353   0.11733548 -64.82112885]
 [  0.07958213

157
0.504538833853
[[  1.22653949   0.19375128 -99.92366028]
 [  0.11913868   0.89475572 -17.46327209]]
158
0.476576098813
[[   1.22753394    0.19477351 -100.39276886]
 [   0.11974423    0.89416087  -17.54728889]]
159
0.476644712706
[[   1.22844446    0.19591875 -100.86019897]
 [   0.12036815    0.89356834  -17.64052582]]
160
0.435648766732
[[   1.22940493    0.19680382 -101.29363251]
 [   0.120947      0.89280558  -17.68439102]]
161
0.400452698659
[[   1.23043764    0.19749732 -101.69332123]
 [   0.12147193    0.89200228  -17.70909119]]
162
0.377618030227
[[   1.23134935    0.19827335 -102.07062531]
 [   0.12191813    0.89121461  -17.72437286]]
163
0.359891716531
[[   1.23220754    0.19898494 -102.43048096]
 [   0.12239701    0.89033163  -17.72898674]]
164
0.382626658528
[[   1.23314965    0.19962861 -102.81240845]
 [   0.12295449    0.88957566  -17.75208473]]
165
0.373288008673
[[   1.23404717    0.20027329 -103.17708588]
 [   0.12358388    0.88903058  -17.83178902]]
166
0.4017670671

232
0.53273408906
[[   1.29694307    0.27147269 -134.55586243]
 [   0.16087046    0.85002899  -22.28196907]]
233
0.499205527117
[[   1.29781032    0.27276421 -135.05090332]
 [   0.16131115    0.84952867  -22.34630394]]
234
0.493421791422
[[   1.29866922    0.27399531 -135.53347778]
 [   0.16185902    0.84905779  -22.44919586]]
235
0.459593949856
[[   1.29950917    0.27509823 -135.99282837]
 [   0.16235746    0.84824705  -22.46383667]]
236
0.480265215046
[[   1.30038464    0.27622873 -136.4703064 ]
 [   0.16285919    0.84760612  -22.5154438 ]]
237
0.467133361806
[[   1.30123687    0.27735505 -136.93305969]
 [   0.16331881    0.84711456  -22.57929611]]
238
0.475822050778
[[   1.30204666    0.27859932 -137.40718079]
 [   0.16387765    0.84639859  -22.61948586]]
239
0.456411173335
[[   1.30291522    0.27967736 -137.85374451]
 [   0.16451138    0.84582478  -22.71379662]]
240
0.445851241237
[[   1.30385244    0.28064388 -138.29620361]
 [   0.16510881    0.84514004  -22.7687149 ]]
241
0.47531

307
0.455064666096
[[   1.36110616    0.3613534  -169.21458435]
 [   0.20054187    0.79336303  -24.37815475]]
308
0.470076209087
[[   1.36192393    0.36255908 -169.67422485]
 [   0.20109178    0.79285842  -24.47661781]]
309
0.391382274857
[[   1.36268497    0.3635267  -170.06257629]
 [   0.20163147    0.79219788  -24.52526474]]
310
0.368558109587
[[   1.36349118    0.3643657  -170.43029785]
 [   0.20218542    0.79140288  -24.55013657]]
311
0.415494555726
[[   1.36431885    0.36543727 -170.84362793]
 [   0.20267014    0.7903772   -24.50785828]]
312
0.451405987812
[[   1.36512411    0.36668676 -171.29502869]
 [   0.20319565    0.78951085  -24.50715637]]
313
0.450001645893
[[   1.36594331    0.36787528 -171.740448  ]
 [   0.20373458    0.78835648  -24.44309807]]
314
0.473789643467
[[   1.36682141    0.36910617 -172.20967102]
 [   0.20433551    0.78763837  -24.50867844]]
315
0.475028030562
[[   1.36772454    0.37035242 -172.68276978]
 [   0.20481554    0.78704417  -24.55144882]]
316
0.4886

382
0.422735421204
[[   1.41583073    0.4351466  -198.08407593]
 [   0.23688816    0.73144794  -25.05506706]]
383
0.447763489671
[[   1.41649139    0.43636248 -198.52043152]
 [   0.2373953     0.73098028  -25.15547752]]
384
0.428805000194
[[   1.41716766    0.4375785  -198.94923401]
 [   0.23784767    0.73015648  -25.15358734]]
385
0.415539935714
[[   1.41787076    0.43874788 -199.36476135]
 [   0.23833589    0.72924942  -25.15502548]]
386
0.413744247809
[[   1.41858125    0.43984789 -199.77571106]
 [   0.23882432    0.72814697  -25.10707092]]
387
0.362566949193
[[   1.41928792    0.44076529 -200.13142395]
 [   0.23928927    0.72694063  -25.03689957]]
388
0.372381958346
[[   1.41992891    0.44174671 -200.50242615]
 [   0.23974888    0.72596443  -25.00485992]]
389
0.380637948278
[[   1.42059231    0.44271785 -200.88108826]
 [   0.24020277    0.72524875  -25.04357719]]
390
0.419818207763
[[   1.42127717    0.44387093 -201.29771423]
 [   0.24064781    0.72414672  -24.99195099]]
391
0.4381

457
0.391484537876
[[   1.46444428    0.51489061 -225.85395813]
 [   0.27485311    0.65930891  -25.14829445]]
458
0.357747030584
[[   1.46507192    0.51601791 -226.20866394]
 [   0.27539632    0.65793854  -25.10181618]]
459
0.33103454922
[[   1.465729      0.51684058 -226.51852417]
 [   0.27584857    0.65763754  -25.21828842]]
460
0.272433958813
[[   1.46638131    0.51735878 -226.77932739]
 [   0.2764416     0.65685147  -25.29705238]]
461
0.395396142118
[[   1.46693385    0.51871258 -227.17448425]
 [   0.27699763    0.65590179  -25.31085396]]
462
0.303645502779
[[   1.4675343     0.51957881 -227.4781189 ]
 [   0.2774671     0.65497398  -25.3080616 ]]
463
0.318626680898
[[   1.46819043    0.5203777  -227.79145813]
 [   0.2780647     0.65350181  -25.25031281]]
464
0.364213274465
[[   1.46884549    0.52143073 -228.15411377]
 [   0.27859792    0.6526978   -25.28398323]]
465
0.368437595487
[[   1.46948075    0.52250516 -228.52067566]
 [   0.27905032    0.65168911  -25.24689674]]
466
0.39548

## Krunal Pics

In [ ]:
best_param = np.float32([[  1.00919437e+00,   2.18628347e-03,  -5.14446564e+01],
       [ -2.68084626e-03,   1.00579178e+00,  -1.56225622e+00]])

In [ ]:
my_param = np.float32([[  1.01574802e+00,  -7.87401572e-04,   4.01141739e+01],
       [  7.87401572e-03,   9.82939661e-01,   2.14041996e+00]])

In [7]:
left = cv2.imread("./images/room_left.jpg")
right = cv2.imread("./images/room_right.jpg")
# img = cv2.imread("./krunal_pics/room_left.jpg")

In [ ]:
img.shape

In [ ]:
left = img[0:400,0:500]

In [ ]:
right = img[0:400,5:505]

In [8]:
show_images([("left",left),("right",right)])

In [ ]:
# point_left = np.float32([[498,276],[300,186],[561,247],[578,361]])
# point_right = np.float32([[451,275],[256,185],[513,245],[531,357]])

In [9]:
gray_right = cv2.cvtColor(right,cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(gray_right,15,0.01,10)
corners = np.int0(corners)
corners = np.int0(corners).squeeze()

In [ ]:
affine_transform = get_transform(right,left,corners)

In [22]:
affine_transform = transform

In [ ]:
affine_transform = cv2.getAffineTransform(point_right[:3],point_left[:3])
# persp_transform = cv2.getPerspectiveTransform(point_right,point_left)

In [24]:
left_affine = cv2.warpAffine(right,affine_transform,(right.shape[1],right.shape[0]))
# left_persp = cv2.warpPerspective(right,persp_transform,(right.shape[1]+left.shape[1],right.shape[0]))

In [ ]:
#left_affine[0:left.shape[0],0:left.shape[1]] = left
# left_persp[0:left.shape[0],0:left.shape[1]] = left

In [ ]:
# show_images([("left",left),("right",right),("affine",left_affine),("persp",left_persp)])

In [25]:
show_images([("left",left),("right",right),("affine",left_affine)])

In [ ]:
for i in corners:
    x,y = i
    cv2.circle(right,(x,y),3,255,-1)